## Server run notebook

This is not the main notebook. See `instructions.ipynb`.

I'll use this notebook to record and check things along the way.

In [1]:
!nvidia-smi

Mon Dec 22 15:45:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:61:00.0 Off |                    0 |
| N/A   39C    P0             72W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## hit this problem

Doing the first test run. Looks like https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip is no longer accessible:

In [3]:
!tail -200 /home/ubuntu/mynanochat2/run_outputs/base_train_output_001.txt

[rank2]:     result = func(*args)
[rank2]:   File "/usr/lib/python3.10/urllib/request.py", line 643, in http_error_default
[rank2]:     raise HTTPError(req.full_url, code, msg, hdrs, fp)
[rank2]: urllib.error.HTTPError: HTTP Error 403: Forbidden
downloading https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip...
[rank7]: Traceback (most recent call last):
[rank7]:   File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
[rank7]:     return _run_code(code, main_globals, None,
[rank7]:   File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
[rank7]:     exec(code, run_globals)
[rank7]:   File "/home/ubuntu/learn-nanochat/my_nanochat/scripts/my_base_train.py", line 205, in <module>
[rank7]:     results = evaluate_model(orig_model, tokenizer, device, max_per_task=core_metric_max_per_task)
[rank7]:   File "/home/ubuntu/learn-nanochat/my_nanochat/scripts/my_base_eval.py", line 41, in evaluate_model
[rank7]:     download_file_with_lock(EVAL_BUNDLE_URL, 'eval_

### repeating test run

...after scp'ing base_eval from my laptop

logging to `base_train_output_002.txt`

In [4]:
!tail -25 /home/ubuntu/mynanochat2/run_outputs/base_train_output_002.txt

wandb:            train/lrm ▁
wandb:            train/mfu ▁
wandb:    train/tok_per_sec ▁
wandb:                   +1 ...
wandb: 
wandb: Run summary:
wandb:          core_metric -0.02267
wandb:                 step 10
wandb: total_training_flops 63331869759897600
wandb:  total_training_time 0
wandb:             train/dt 22.59089
wandb:      train/grad_norm 0.53604
wandb:           train/loss 11.09035
wandb:            train/lrm 1
wandb:            train/mfu 3.54326
wandb:    train/tok_per_sec 23207
wandb:                   +1 ...
wandb: 
wandb: 🚀 View run challenge-38-1 at: https://wandb.ai/ericsilberstein-self/my-nanochat/runs/0ie1fin2
wandb: ⭐️ View project at: https://wandb.ai/ericsilberstein-self/my-nanochat
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20251222_155927-0ie1fin2/logs
[W1222 16:01:23.002992110 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (

### confirm can load checkpoint

In [7]:
import torch
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_common import get_base_dir, autodetect_device_type, compute_init
from my_nanochat.my_checkpoint_manager import load_model

In [8]:
device_type = autodetect_device_type() 
_, _, _, _, device = compute_init(device_type)
model, tokenizer, meta_data = load_model('base', model_tag='d32', device=device, phase='eval')
autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=torch.bfloat16) if device_type == "cuda" else nullcontext()

Autodetected device type: cuda


/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


loading the model from /home/ubuntu/mynanochat2/base_checkpoints/d32 with step 10
Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 32, 'n_head': 16, 'n_kv_head': 16, 'n_embd': 2048}


In [9]:
with autocast_ctx:
    tokens = list(model.generate(tokenizer.encode("First take a right on Main St.", prepend=tokenizer.get_bos_token_id()), max_tokens=10))
tokens

[1006, 345, 400, 311, 4236, 3328, 281, 261, 24142, 4398]

In [10]:
tokenizer.decode(tokens)

' We are can be introduced examples of the utmost background'

that looks ok, also check wandb

look ok, do real run

### full run followed by base_loss followed by base_eval

In [1]:
!tail -100 $RUN_OUTPUTS_DIR/base_train_output_004.txt

step 71658/71680 (99.97%) | loss: 2.372278 | grad norm: 0.1515 | lrm: 0.00 | dt: 1705.46ms | tok/sec: 307,416 | mfu: 46.93 | total time: 1961.10m
step 71659/71680 (99.97%) | loss: 2.378669 | grad norm: 0.1473 | lrm: 0.00 | dt: 2489.89ms | tok/sec: 210,566 | mfu: 32.15 | total time: 1961.14m
step 71660/71680 (99.97%) | loss: 2.409759 | grad norm: 0.1564 | lrm: 0.00 | dt: 1513.21ms | tok/sec: 346,474 | mfu: 52.90 | total time: 1961.17m
step 71661/71680 (99.97%) | loss: 2.400042 | grad norm: 0.1685 | lrm: 0.00 | dt: 1523.07ms | tok/sec: 344,230 | mfu: 52.56 | total time: 1961.19m
step 71662/71680 (99.97%) | loss: 2.398359 | grad norm: 0.1515 | lrm: 0.00 | dt: 1516.79ms | tok/sec: 345,656 | mfu: 52.77 | total time: 1961.22m
step 71663/71680 (99.98%) | loss: 2.397501 | grad norm: 0.1507 | lrm: 0.00 | dt: 1519.49ms | tok/sec: 345,041 | mfu: 52.68 | total time: 1961.24m
step 71664/71680 (99.98%) | loss: 2.383298 | grad norm: 0.1942 | lrm: 0.00 | dt: 1520.90ms | tok/sec: 344,722 | mfu: 52.63 |

In [3]:
!cat $RUN_OUTPUTS_DIR/base_loss_output_001.txt

Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'

In [8]:
!cat $RUN_OUTPUTS_DIR/base_eval_output.001.txt

Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'

### also cat reports

In [14]:
!cat $NANOCHAT_BASE_DIR/report/header.md

# nanochat training report

Generated: 2025-12-22 16:07:10

## Environment

### Git Information
- Branch: master
- Commit: bc88d72 (dirty)
- Message: getting ready to work on gpu machine for challenge 38: train d32

### Hardware
- Platform: Linux
- CPUs: 104 cores (208 logical)
- Memory: 1771.7 GB
- GPUs: 8x NVIDIA H100 80GB HBM3
- GPU Memory: 633.5 GB total
- CUDA Version: 12.8

### Software
- Python: 3.10.12
- PyTorch: 2.9.0+cu128

Run started: 2025-12-22 16:07:10

--



In [10]:
!cat $NANOCHAT_BASE_DIR/report/base-model-training.md

## Base model training
timestamp: 2025-12-24 02:02:38

- run: challenge-38-4
- device_type: 
- depth: 32
- max_seq_len: 2048
- num_iterations: -1
- target_param_data_ratio: 20
- device_batch_size: 8
- total_batch_size: 524,288
- embedding_lr: 0.2000
- unembedding_lr: 0.0040
- weight_decay: 0.0000
- matrix_lr: 0.0200
- grad_clip: 1.0000
- warmup_ratio: 0.0000
- warmdown_ratio: 0.2000
- final_lr_frac: 0.0000
- eval_every: 250
- eval_tokens: 10,485,760
- core_metric_every: 2000
- core_metric_max_per_task: 500
- sample_every: 2000
- model_tag: 
- Number of parameters: 1,879,048,192
- Number of FLOPs per token: 1.207960e+10
- Calculated number of iterations: 71,680
- Number of training tokens: 37,580,963,840
- Tokens : Params ratio: 20.0000
- DDP world size: 8
- warmup_ratio: 0.0000
- warmdown_ratio: 0.2000
- final_lr_frac: 0.0000
- Minimum validation bpb: 0.7233
- Final validation bpb: 0.7233
- CORE metric estimate: 0.3050
- MFU %: 53.03%
- Total training flops: 4.539628e+20
- Total traini

In [11]:
!cat $NANOCHAT_BASE_DIR/report/base-model-loss.md

## Base model loss
timestamp: 2025-12-24 02:05:23

- train bpb: 0.7271
- val bpb: 0.7235
- sample 0: <|bos|>The capital of France is Paris. It is the largest city in the country and the second largest in Europe
- sample 1: <|bos|>The chemical symbol of gold is Au. Gold is a chemical element with the atomic number 79 and the atomic
- sample 2: <|bos|>If yesterday was Friday, then tomorrow will be Saturday. If today is Monday, then tomorrow will be Tuesday. If today is
- sample 3: <|bos|>The opposite of hot is cold. The opposite of hot is cold. The opposite of hot is cold.
- sample 4: <|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune
- sample 5: <|bos|>My favorite color is blue. I love the color blue. I love the color blue. I love
- sample 6: <|bos|>If 5*x + 3 = 13, then x is equal to 3.
If 5*x + 3 = 13



In [13]:
!cat $NANOCHAT_BASE_DIR/report/base-model-evaluation.md

## Base model evaluation
timestamp: 2025-12-24 02:12:40

- Model: base_model (step 71680)
- CORE metric: 0.2993
- hellaswag_zeroshot: 0.4429
- jeopardy: 0.2976
- bigbench_qa_wikidata: 0.6202
- arc_easy: 0.6302
- arc_challenge: 0.2457
- copa: 0.4200
- commonsense_qa: 0.0469
- piqa: 0.4788
- openbook_qa: 0.1840
- lambada_openai: 0.4910
- hellaswag: 0.4518
- winograd: 0.4945
- winogrande: 0.1997
- bigbench_dyck_languages: 0.0780
- agi_eval_lsat_ar: 0.0598
- bigbench_cs_algorithms: 0.3720
- bigbench_operators: 0.1762
- bigbench_repeat_copy_logic: 0.0312
- squad: 0.3996
- coqa: 0.2937
- boolq: -0.0068
- bigbench_language_identification: 0.1776



### mid train and chat eval run output and reports

I ran these on the 8 GPU server and then copied the run output and reports to my laptop.

In [3]:
!cat ../files_copied_from_servers/challenge-38-train-d32/run_outputs/mid_train_output_002.txt

overriding model_tag = d32
overriding device_batch_size = 8
overriding run = challenge-38-6
user_config: {'run': 'challenge-38-6', 'device_type': '', 'dtype': 'bfloat16', 'num_iterations': -1, 'max_seq_len': 2048, 'device_batch_size': 8, 'unembedding_lr': 0.004, 'embedding_lr': 0.2, 'matrix_lr': 0.02, 'init_lr_frac': 1.0, 'weight_decay': 0.0, 'eval_every': 150, 'eval_tokens': 10485760, 'total_batch_size': 524288, 'dry_run': 0}
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-amper

In [4]:
!cat ../files_copied_from_servers/challenge-38-train-d32/run_outputs/mid_chat_eval_output_001.txt

Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'

In [5]:
!cat ../files_copied_from_servers/challenge-38-train-d32/report/midtraining.md

## Midtraining
timestamp: 2025-12-29 16:34:21

- run: challenge-38-6
- device_type: 
- dtype: bfloat16
- num_iterations: -1
- max_seq_len: 2048
- device_batch_size: 8
- unembedding_lr: 0.0040
- embedding_lr: 0.2000
- matrix_lr: 0.0200
- init_lr_frac: 1.0000
- weight_decay: 0.0000
- eval_every: 150
- eval_tokens: 10,485,760
- total_batch_size: 524,288
- dry_run: 0
- Number of iterations: 810
- DDP world size: 8
- Minimum validation bpb: 0.3392



In [6]:
!cat ../files_copied_from_servers/challenge-38-train-d32/report/chat-evaluation-mid.md

## Chat evaluation mid
timestamp: 2025-12-29 16:44:49

- source: mid
- task_name: None
- dtype: bfloat16
- temperature: 0.0000
- max_new_tokens: 512
- num_samples: 1
- top_k: 50
- batch_size: 8
- model_tag: d32
- step: None
- max_problems: None
- print_failed: False
- device_type: 
- ARC-Easy: 0.6507
- ARC-Challenge: 0.4650
- MMLU: 0.3909
- GSM8K: 0.1077
- HumanEval: 0.0854
- SpellingBee: 0.9922
- ChatCORE metric: 0.3657



### sft train and chat eval run output and reports

I ran these on the 8 GPU server and then copied the run output and reports to my laptop.

In [8]:
!cat ../files_copied_from_servers/challenge-38-train-d32/run_outputs/sft_train_output_001.txt

Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'

In [9]:
!cat ../files_copied_from_servers/challenge-38-train-d32/run_outputs/sft_chat_eval_output_001.txt

Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'

In [10]:
!cat ../files_copied_from_servers/challenge-38-train-d32/report/chat-sft.md

## Chat SFT
timestamp: 2025-12-29 16:51:52

- run: challenge-38-7
- source: mid
- device_type: 
- dtype: bfloat16
- device_batch_size: 4
- num_epochs: 1
- num_iterations: -1
- max_data_tokens: -1
- target_examples_per_step: 32
- unembedding_lr: 0.0040
- embedding_lr: 0.2000
- matrix_lr: 0.0200
- weight_decay: 0.0000
- init_lr_frac: 0.0200
- eval_every: 100
- eval_steps: 100
- eval_metrics_every: 200
- eval_metrics_max_problems: 1024
- Training rows: 22,439
- Number of iterations: 701
- Training loss: 0.4471
- Validation loss: 0.8385



In [11]:
!cat ../files_copied_from_servers/challenge-38-train-d32/report/chat-evaluation-sft.md

## Chat evaluation sft
timestamp: 2025-12-29 17:02:10

- source: sft
- task_name: None
- dtype: bfloat16
- temperature: 0.0000
- max_new_tokens: 512
- num_samples: 1
- top_k: 50
- batch_size: 8
- model_tag: d32
- step: None
- max_problems: None
- print_failed: False
- device_type: 
- ARC-Easy: 0.6650
- ARC-Challenge: 0.4616
- MMLU: 0.3971
- GSM8K: 0.1334
- HumanEval: 0.0793
- SpellingBee: 0.9922
- ChatCORE metric: 0.3727



### "final" report (excludes reinforcement learning)

In [12]:
!cat ../files_copied_from_servers/challenge-38-train-d32/report/report.md

# nanochat training report

Generated: 2025-12-22 16:07:10

## Environment

### Git Information
- Branch: master
- Commit: bc88d72 (dirty)
- Message: getting ready to work on gpu machine for challenge 38: train d32

### Hardware
- Platform: Linux
- CPUs: 104 cores (208 logical)
- Memory: 1771.7 GB
- GPUs: 8x NVIDIA H100 80GB HBM3
- GPU Memory: 633.5 GB total
- CUDA Version: 12.8

### Software
- Python: 3.10.12
- PyTorch: 2.9.0+cu128

Run started: 2025-12-22 16:07:10

--

## Base model training
timestamp: 2025-12-24 02:02:38

- run: challenge-38-4
- device_type: 
- depth: 32
- max_seq_len: 2048
- num_iterations: -1
- target_param_data_ratio: 20
- device_batch_size: 8
- total_batch_size: 524,288
- embedding_lr: 0.2000
- unembedding_lr: 0.0040
- weight_decay: 0.0000
- matrix_lr: 0.0200
- grad_clip: 1.0000
- warmup_ratio: 0.0000
- warmdown_ratio: 0.2000
- final_lr_frac: 0.0000
- eval_every: 250
- eval_tokens: 10,485,760
- core_metric_every: 2000
- core_metric_max_per_task: 500
- sample_eve

### rl train and chat eval run output and reports

I ran these on the 8 GPU server and then copied the run output and reports to my laptop.

The chat eval is based on the step 360 checkpoint.

In [13]:
!cat ../files_copied_from_servers/challenge-38-train-d32/run_outputs/rl_train_output_001.txt

overriding source = sftAutodetected device type: cudaAutodetected device type: cudaAutodetected device type: cuda
Autodetected device type: cudaAutodetected device type: cudaAutodetected device type: cuda


overriding model_tag = d32



Autodetected device type: cudaoverriding run = challenge-38-8

user_config: {'run': 'challenge-38-8', 'source': 'sft', 'dtype': 'bfloat16', 'device_type': '', 'num_steps': -1, 'device_batch_size': 8, 'examples_per_step': 16, 'num_samples': 16, 'max_new_tokens': 256, 'temperature': 1.0, 'top_k': 50, 'unembedding_lr': 0.004, 'embedding_lr': 0.2, 'matrix_lr': 0.02, 'weight_decay': 0.0, 'init_lr_frac': 0.05, 'num_epochs': 1, 'save_every': 60, 'eval_every': 60, 'eval_examples': 400}
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp3

In [14]:
!cat ../files_copied_from_servers/challenge-38-train-d32/run_outputs/rl_chat_eval_output_001.txt

Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
Autodetected device type: cuda
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda

In [16]:
!cat ../files_copied_from_servers/challenge-38-train-d32/report/chat-evaluation-rl.md

## Chat evaluation rl
timestamp: 2025-12-31 17:00:49

- source: rl
- task_name: None
- dtype: bfloat16
- temperature: 0.0000
- max_new_tokens: 512
- num_samples: 1
- top_k: 50
- batch_size: 8
- model_tag: d32
- step: 360
- max_problems: None
- print_failed: False
- device_type: 
- ARC-Easy: 0.6612
- ARC-Challenge: 0.4753
- MMLU: 0.3950
- GSM8K: 0.1926
- HumanEval: 0.0915
- SpellingBee: 0.9688
- ChatCORE metric: 0.3824

